In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Flatten, TimeDistributed, Dropout, LSTMCell, RNN, Bidirectional, Concatenate, Layer,GRU,LeakyReLU
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.utils import tf_utils
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model,load_model

import unicodedata
import re
import numpy as np
import os
import time
import shutil

import pandas as pd
import numpy as np
import string, os 
tf.__version__

In [ ]:
data=pd.read_csv('../input/hinglish-data/valid_training_sentences_n_grams_f_3_freqcount5000.csv')
data.head()

In [ ]:
len(data)

In [ ]:
corpus=data['sentences']#[:20000]

In [ ]:


def generate_dataset():
  
     
    output = []
    for line in corpus:
        token_list = line.split(' ')
        for i in range(1, len(token_list)):
            data = []
            x_ngram =  token_list[:i+1]
            x_ngram.insert(0, '<start>')
            x_ngram.append('<end>')
            y_ngram=token_list[i+1:]
            y_ngram.insert(0, '<start>')
            y_ngram.append('<end>')
            data.append(' '.join(x_ngram))
            data.append(' '.join(y_ngram))
            output.append(data)
    print("Dataset prepared with prefix and suffixes for teacher forcing technique")
    dummy_df = pd.DataFrame(output, columns=['input','output'])
    return output, dummy_df 



In [ ]:
class LanguageIndex():
    def __init__(self, lang):
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
        self.create_index()
    def create_index(self):
        for phrase in self.lang:
            self.vocab.update(phrase.split(' '))
        self.vocab = sorted(self.vocab)
        self.word2idx["<pad>"] = 0
        self.idx2word[0] = "<pad>"
#         self.word2idx[" "]=1
#         self.idx2word[1]=" "
        for i,word in enumerate(self.vocab):
            self.word2idx[word] = i + 1
            self.idx2word[i+1] = word

def max_length(t):
    return max(len(i) for i in t)

def load_dataset():
    pairs,df = generate_dataset()
    out_lang = LanguageIndex(sp for en, sp in pairs)
    in_lang = LanguageIndex(en for en, sp in pairs)
    input_data = [[in_lang.word2idx[s] for s in en.split(' ')] for en, sp in pairs]
    output_data = [[out_lang.word2idx[s] for s in sp.split(' ')] for en, sp in pairs]

    max_length_in, max_length_out = max_length(input_data), max_length(output_data)
    input_data = tf.keras.preprocessing.sequence.pad_sequences(input_data, maxlen=max_length_in, padding="post")
    output_data = tf.keras.preprocessing.sequence.pad_sequences(output_data, maxlen=max_length_out, padding="post")
    return input_data, output_data, in_lang, out_lang, max_length_in, max_length_out, df

In [ ]:
input_data, teacher_data, input_lang, target_lang, len_input, len_target, df  = load_dataset()


target_data = [[teacher_data[n][i+1] for i in range(len(teacher_data[n])-1)] for n in range(len(teacher_data))]
target_data = tf.keras.preprocessing.sequence.pad_sequences(target_data, maxlen=len_target, padding="post")
target_data = target_data.reshape((target_data.shape[0], target_data.shape[1], 1))

# Shuffle all of the data in unison. This training set has the longest (e.g. most complicated) data at the end,
# so a simple Keras validation split will be problematic if not shuffled.

p = np.random.permutation(len(input_data))
input_data = input_data[p]
teacher_data = teacher_data[p]
target_data = target_data[p]

In [ ]:
len_target

In [ ]:
df

In [ ]:
len(input_lang.word2idx) 
#input_lang.word2idx

In [ ]:
pd.set_option('display.max_colwidth', -1)
BUFFER_SIZE = len(input_data)
BATCH_SIZE = 128
embedding_dim = 100
units = 512
vocab_in_size = len(input_lang.word2idx)
vocab_out_size = len(target_lang.word2idx)
df.iloc[60:65]

In [ ]:

# # Create the Encoder layers first.
# encoder_inputs = Input(shape=(len_input,))
# encoder_emb = Embedding(input_dim=vocab_in_size, output_dim=embedding_dim)

# # Use this if you dont need Bidirectional LSTM
# # encoder_lstm = CuDNNLSTM(units=units, return_sequences=True, return_state=True)
# # encoder_out, state_h, state_c = encoder_lstm(encoder_emb(encoder_inputs))

# encoder_lstm = Bidirectional(LSTM(units=units, return_sequences=True, return_state=True))
# encoder_out, fstate_h, fstate_c, bstate_h, bstate_c = encoder_lstm(encoder_emb(encoder_inputs))
# state_h = Concatenate()([fstate_h,bstate_h])
# state_c = Concatenate()([bstate_h,bstate_c])
# encoder_states = [state_h, state_c]


# # Now create the Decoder layers.
# decoder_inputs = Input(shape=(None,))
# decoder_emb = Embedding(input_dim=vocab_out_size, output_dim=embedding_dim)
# decoder_lstm = LSTM(units=units*2, return_sequences=True, return_state=True)
# decoder_lstm_out, _, _ = decoder_lstm(decoder_emb(decoder_inputs), initial_state=encoder_states)
# # Two dense layers added to this model to improve inference capabilities.
# decoder_d1 = Dense(units)
# decoder_d11=LeakyReLU(alpha=0.05)
# decoder_d2 = Dense(vocab_out_size, activation="softmax")
# decoder_out = decoder_d2(Dropout(rate=.2)(decoder_d1(Dropout(rate=.2)(decoder_lstm_out))))


# # Finally, create a training model which combines the encoder and the decoder.
# # Note that this model has three inputs:
# model = Model(inputs = [encoder_inputs, decoder_inputs], outputs= decoder_out)

# # We'll use sparse_categorical_crossentropy so we don't have to expand decoder_out into a massive one-hot array.
# # Adam is used because it's, well, the best.

# model.compile(optimizer=tf.optimizers.Adam(), loss="sparse_categorical_crossentropy", metrics=['sparse_categorical_accuracy'])
# model.summary()

In [ ]:

# Create the Encoder layers first.
encoder_inputs = Input(shape=(len_input,))
encoder_emb = Embedding(input_dim=vocab_in_size, output_dim=embedding_dim)

# Use this if you dont need Bidirectional LSTM
# encoder_lstm = CuDNNLSTM(units=units, return_sequences=True, return_state=True)
# encoder_out, state_h, state_c = encoder_lstm(encoder_emb(encoder_inputs))

encoder_lstm = Bidirectional(LSTM(units=units, return_sequences=True, return_state=True))
encoder_out, fstate_h, fstate_c, bstate_h, bstate_c = encoder_lstm(encoder_emb(encoder_inputs))
state_h = Concatenate()([fstate_h,bstate_h])
state_c = Concatenate()([bstate_h,bstate_c])
encoder_states = [state_h, state_c]


# Now create the Decoder layers.
decoder_inputs = Input(shape=(None,))
decoder_emb = Embedding(input_dim=vocab_out_size, output_dim=embedding_dim)
decoder_lstm = LSTM(units=units*2, return_sequences=True, return_state=True)
decoder_lstm_out, _, _ = decoder_lstm(decoder_emb(decoder_inputs), initial_state=encoder_states)
# Two dense layers added to this model to improve inference capabilities.
decoder_d1 = Dense(units)
decoder_d2=LeakyReLU()
decoder_d3 = Dense(vocab_out_size, activation="softmax")
decoder_out = decoder_d3(Dropout(rate=.2)(decoder_d2(decoder_d1(Dropout(rate=.2)(decoder_lstm_out)))))


# Finally, create a training model which combines the encoder and the decoder.
# Note that this model has three inputs:
model = Model(inputs = [encoder_inputs, decoder_inputs], outputs= decoder_out)

# We'll use sparse_categorical_crossentropy so we don't have to expand decoder_out into a massive one-hot array.
# Adam is used because it's, well, the best.

model.compile(optimizer=tf.optimizers.Adam(), loss="sparse_categorical_crossentropy", metrics=['sparse_categorical_accuracy'])
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint

# Note, we use 20% of our data for validation.
epochs = 5

history = model.fit([input_data, teacher_data], target_data,
                 batch_size= BATCH_SIZE,
                 epochs=epochs,
                 validation_split=0.2)

In [ ]:
# encoder_model.save('model11/encoder_model.h5')
# inf_model.save('model11/inf_model.h5')


# from keras.models import model_from_json
# from keras.models import load_model


# inf_model_json= inf_model.to_json()
# encoder_model_json= encoder_model.to_json()



# with open("model11/inf_model.json", "w") as json_file:
#     json_file.write(inf_model_json)
# with open("model11/encoder_model.json", "w") as json_file:
#     json_file.write(encoder_model_json)

# #serialize weights to HDF5
# inf_model.save_weights("model11/inf_model_weights.h5")
# encoder_model.save_weights("model11/encoder_model_weights.h5")

In [ ]:
# Create the encoder model from the tensors we previously declared.
encoder_model = Model(encoder_inputs, [encoder_out, state_h, state_c])

# Generate a new set of tensors for our new inference decoder. Note that we are using new tensors, 
# this does not preclude using the same underlying layers that we trained on. (e.g. weights/biases).

inf_decoder_inputs = Input(shape=(None,), name="inf_decoder_inputs")
# We'll need to force feed the two state variables into the decoder each step.
state_input_h = Input(shape=(units*2,), name="state_input_h")
state_input_c = Input(shape=(units*2,), name="state_input_c")
decoder_res, decoder_h, decoder_c = decoder_lstm(
    decoder_emb(inf_decoder_inputs), 
    initial_state=[state_input_h, state_input_c])
inf_decoder_out = decoder_d3(decoder_d2(decoder_d1(decoder_res)))
inf_model = Model(inputs=[inf_decoder_inputs, state_input_h, state_input_c], 
                  outputs=[inf_decoder_out, decoder_h, decoder_c])

In [ ]:
##greedy approch

# Given an input string, an encoder model (infenc_model) and a decoder model (infmodel),
def translate_greedy(input_sentence, infenc_model, infmodel):
    sv = sentence_to_vector(input_sentence, input_lang)
    #print(sv)
    sv = sv.reshape(1,len(sv))
    [emb_out, sh, sc] = infenc_model.predict(x=sv)
    
    i = 0
    start_vec = target_lang.word2idx["<start>"]
    stop_vec = target_lang.word2idx["<end>"]
    
    cur_vec = np.zeros((1,1))
    cur_vec[0,0] = start_vec
    cur_word = "<start>"
    output_sentence = ""

    while cur_word != "<end>" and i < (len_target-1):
        i += 1
        if cur_word != "<start>":
            output_sentence = output_sentence + " " + cur_word
        x_in = [cur_vec, sh, sc]
        [nvec, sh, sc] = infmodel.predict(x=x_in)
        cur_vec[0,0] = np.argmax(nvec[0,0])
        cur_word = target_lang.idx2word[np.argmax(nvec[0,0])]
    return output_sentence

In [ ]:
# Beam search decoding
import numpy as np

def predicts(xin,infmodel,prev_p):
    
    
    [v, sh, sc] = infmodel.predict(x=xin)
    w=list((-v[0,0]).argsort()[0:3])
    p=sorted(v[0,0],reverse=True)[0:3]
    max_p_arg=np.argmax([i*np.log(prev_p) for i in p])
    max_p=max([i*np.log(prev_p) for i in p])
    
    
    return [np.array(w[max_p_arg]).reshape(1,1),sh,sc],max_p
    

def Beam_search_decoder(inf,infmodel):
    
    start_vec = target_lang.word2idx["<start>"]
    stop_vec = target_lang.word2idx["<end>"]
    
    cur_vec = np.zeros((1,1))
    cur_vec[0,0] = start_vec
    cur_word = "<start>"
    x_in = [cur_vec,inf[1],inf[2]]
    [nvec, sh, sc] = infmodel.predict(x=x_in)
    
    v=nvec[0][0]
    li=[]
    probs=[]
    lis=[]

    w1=(-v).argsort()[0]
    p1=sorted(v,reverse=True)[0]
    li.append([[w1],p1])

    w2=(-v).argsort()[1]
    p2=sorted(v,reverse=True)[1]
    li.append([[w2],p2])

    w3=(-v).argsort()[2]
    p3=sorted(v,reverse=True)[2]
    li.append([[w3],p3])

    
    
    for j in li:
        cur_vec[0,0]=j[0][0]
        
        p=j[1]
        i=1
        cur_word=''
        xin=[cur_vec, sh, sc]
       
        while cur_word != "<end>" and i < (len_target-1):
            xin,p=predicts(xin,infmodel,p)
            
            cur_word = target_lang.idx2word[xin[0][0,0]]
            j[0].append(xin[0][0,0])
            
            
            i+=1
            
            if cur_word == "<end>" or i >= (len_target-1):
                probs.append(p)
        
        lis.append(j[0])
    st=''
    for i in lis[np.argmax(probs)][:-1]:
        st=st+' '+target_lang.idx2word[i]
            
    return st
        
          
        
    


def translate_beam(input_sentence, infenc_model, infmodel):
    sv = sentence_to_vector(input_sentence, input_lang)
    sv = sv.reshape(1,len(sv))
    [emb_out, sh, sc] = infenc_model.predict(x=sv)
    
    lis=Beam_search_decoder([emb_out, sh, sc],infmodel)
    return lis

In [ ]:
# Output is 1-D: [timesteps/words]
def sentence_to_vector(sentence, lang):

    pre = sentence
    vec = np.zeros(len_input)
    sentence_list = [lang.word2idx[s] for s in pre.split(' ')]
    for i,w in enumerate(sentence_list):
        vec[i] = w
    return vec  
    
#Note that only words that we've trained the model on will be available, otherwise you'll get an error.


test = [
    'here',
  'how are',
   'have a nice',
   'bhai kya',
  'kaisa h',
    'chal',
      'or kya',
   'let',
  'good',
  'happy',
    'Let me',
  'Let me know if',
    'bahut',
      'accha'
]



        
        


output = []  
for t in test:
    t='<start> '+t+' <end>'
    output.append({"Input seq":t.lower(), "Pred. Seq":translate_greedy(t.lower(), encoder_model, inf_model)})
    #print(t)

results_df = pd.DataFrame.from_dict(output) 
results_df

In [ ]:
# Output is 1-D: [timesteps/words]
def sentence_to_vector(sentence, lang):

    pre = sentence
    vec = np.zeros(len_input)
    sentence_list = [lang.word2idx[s] for s in pre.split(' ')]
    for i,w in enumerate(sentence_list):
        vec[i] = w
    return vec  
    
#Note that only words that we've trained the model on will be available, otherwise you'll get an error.


test = [
   'here',
  'how are',
   'have a nice',
   'bhai kya',
  'kaisa h',
    'chal',
      'or kya',
   'let',
  'good',
  'happy',
    'Let me',
  'Let me know if',
    'bahut',
      'accha'
]



        
        

output = []  
for t in test:
    t='<start> '+t+' <end>'
    output.append({"Input seq":t.lower(), "Pred. Seq":translate_beam(t.lower(), encoder_model, inf_model)})
    #print(t)

results_df = pd.DataFrame.from_dict(output) 
results_df

In [ ]:
# from keras.models import model_from_json
# from keras.models import load_model


# inf_model_json= inf_model.to_json()
# encoder_model_json= encoder_model.to_json()



# with open("inf_model.json", "w") as json_file:
#     json_file.write(inf_model_json)
# with open("encoder_model.json", "w") as json_file:
#     json_file.write(encoder_model_json)

# #serialize weights to HDF5
# inf_model.save_weights("inf_model_weights.h5")
# encoder_model.save_weights("encoder_model_weights.h5")

# encoder_model.save('encoder_model.h5')
# inf_model.save('inf_model.h5')

In [ ]:
## using GRU model

# Create the Encoder layers first.
encoder_inputs = Input(shape=(len_input,))
encoder_emb = Embedding(input_dim=vocab_in_size, output_dim=embedding_dim)

# Use this if you dont need Bidirectional LSTM
# encoder_lstm = CuDNNLSTM(units=units, return_sequences=True, return_state=True)
# encoder_out, state_h, state_c = encoder_lstm(encoder_emb(encoder_inputs))

encoder_lstm = Bidirectional(LSTM(units=units, return_sequences=True, return_state=True))
encoder_out, fstate_h, fstate_c, bstate_h, bstate_c = encoder_lstm(encoder_emb(encoder_inputs))
state_h = Concatenate()([fstate_h,bstate_h])
state_c = Concatenate()([bstate_h,bstate_c])
encoder_states = [state_h, state_c]


# Now create the Decoder layers.
decoder_inputs = Input(shape=(None,))
decoder_emb = Embedding(input_dim=vocab_out_size, output_dim=embedding_dim)
decoder_lstm = LSTM(units=units*2, return_sequences=True, return_state=True)
decoder_lstm_out, _, _ = decoder_lstm(decoder_emb(decoder_inputs), initial_state=encoder_states)
# Two dense layers added to this model to improve inference capabilities.
decoder_d1 = Dense(units, activation="relu")
decoder_d2 = Dense(vocab_out_size, activation="softmax")
decoder_out = decoder_d2(Dropout(rate=.2)(decoder_d1(Dropout(rate=.2)(decoder_lstm_out))))


# Finally, create a training model which combines the encoder and the decoder.
# Note that this model has three inputs:
model = Model(inputs = [encoder_inputs, decoder_inputs], outputs= decoder_out)

# We'll use sparse_categorical_crossentropy so we don't have to expand decoder_out into a massive one-hot array.
# Adam is used because it's, well, the best.

model.compile(optimizer=tf.optimizers.Adam(), loss="sparse_categorical_crossentropy", metrics=['sparse_categorical_accuracy'])
model.summary()

In [ ]:

# Create the Encoder layers first.
encoder_inputs = Input(shape=(len_input,))
encoder_emb = Embedding(input_dim=vocab_in_size, output_dim=embedding_dim)

# Use this if you dont need Bidirectional LSTM
# encoder_lstm = CuDNNLSTM(units=units, return_sequences=True, return_state=True)
# encoder_out, state_h, state_c = encoder_lstm(encoder_emb(encoder_inputs))


encoder_lstm = GRU(units=units, return_sequences=True, return_state=True)
encoder_out, state_h= encoder_lstm(encoder_emb(encoder_inputs))
# state_h = Concatenate()([fstate_h,bstate_h])
# state_c = Concatenate()([bstate_h,bstate_c])
encoder_states = state_h


# Now create the Decoder layers.
decoder_inputs = Input(shape=(None,))
decoder_emb = Embedding(input_dim=vocab_out_size, output_dim=embedding_dim)
decoder_lstm = GRU(units=units, return_sequences=True, return_state=True)
decoder_lstm_out, _= decoder_lstm(decoder_emb(decoder_inputs), initial_state=encoder_states)
# Two dense layers added to this model to improve inference capabilities.
decoder_d1 = Dense(units, activation="relu")
decoder_d2 = Dense(vocab_out_size, activation="softmax")
decoder_out = decoder_d2(Dropout(rate=.2)(decoder_d1(Dropout(rate=.2)(decoder_lstm_out))))


# Finally, create a training model which combines the encoder and the decoder.
# Note that this model has three inputs:
model = Model(inputs = [encoder_inputs, decoder_inputs], outputs= decoder_out)

# We'll use sparse_categorical_crossentropy so we don't have to expand decoder_out into a massive one-hot array.
# Adam is used because it's, well, the best.

model.compile(optimizer=tf.optimizers.Adam(), loss="sparse_categorical_crossentropy", metrics=['sparse_categorical_accuracy'])
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint

# Note, we use 20% of our data for validation.
epochs = 1

history = model.fit([input_data, teacher_data], target_data,
                 batch_size= BATCH_SIZE,
                 epochs=epochs,
                 validation_split=0.2)

In [ ]:
# Create the encoder model from the tensors we previously declared.
encoder_model = Model(encoder_inputs, [encoder_out, state_h])

# Generate a new set of tensors for our new inference decoder. Note that we are using new tensors, 
# this does not preclude using the same underlying layers that we trained on. (e.g. weights/biases).

inf_decoder_inputs = Input(shape=(None,), name="inf_decoder_inputs")
# We'll need to force feed the two state variables into the decoder each step.
state_input_h = Input(shape=(units,), name="state_input_h")
#state_input_c = Input(shape=(units*2,), name="state_input_c")
decoder_res, decoder_h = decoder_lstm(
    decoder_emb(inf_decoder_inputs), 
    initial_state=state_input_h)
inf_decoder_out = decoder_d2(decoder_d1(decoder_res))
inf_model = Model(inputs=[inf_decoder_inputs, state_input_h], 
                  outputs=[inf_decoder_out, decoder_h])

In [ ]:
##greedy approch

# Given an input string, an encoder model (infenc_model) and a decoder model (infmodel),
def translate_greedy(input_sentence, infenc_model, infmodel):
    sv = sentence_to_vector(input_sentence, input_lang)
    #print(sv)
    sv = sv.reshape(1,len(sv))
  
    [emb_out, sh] = infenc_model.predict(x=sv)
    
    i = 0
    start_vec = target_lang.word2idx["<start>"]
    stop_vec = target_lang.word2idx["<end>"]
    
    cur_vec = np.zeros((1,1))
    cur_vec[0,0] = start_vec
    cur_word = "<start>"
    output_sentence = ""

    while cur_word != "<end>" and i < (len_target-1):
        i += 1
        if cur_word != "<start>":
            output_sentence = output_sentence + " " + cur_word
        x_in = [cur_vec, sh]
        [nvec, sh] = infmodel.predict(x=x_in)
        cur_vec[0,0] = np.argmax(nvec[0,0])
        cur_word = target_lang.idx2word[np.argmax(nvec[0,0])]
    return output_sentence

In [ ]:
# Output is 1-D: [timesteps/words]
def sentence_to_vector(sentence, lang):

    pre = sentence
    #pre = '<start> ' + pre + ' <end>'
    vec = np.zeros(len_input)
    sentence_list = [lang.word2idx[s] for s in pre.split(' ')]
    for i,w in enumerate(sentence_list):
        vec[i] = w
    return vec  
    
#Note that only words that we've trained the model on will be available, otherwise you'll get an error.


test = [
    'here',
  'how are',
   'have a nice',
   'bhai kya',
  'kaisa ho',
    'chal fir',
      'or kya',
   'ok',
  'good',
  'than',
    'Let me',
  'Let me know if',
    'by',
      'kl'
]



        
        


output = []  
for t in test:
    
    output.append({"Input seq":t.lower(), "Pred. Seq":translate_greedy(t.lower(), encoder_model, inf_model)})
    #print(t)

results_df = pd.DataFrame.from_dict(output) 
results_df

**for bidirectional GRU**

In [ ]:

# Create the Encoder layers first.
encoder_inputs = Input(shape=(len_input,))
encoder_emb = Embedding(input_dim=vocab_in_size, output_dim=embedding_dim)

# Use this if you dont need Bidirectional LSTM
# encoder_lstm = CuDNNLSTM(units=units, return_sequences=True, return_state=True)
# encoder_out, state_h, state_c = encoder_lstm(encoder_emb(encoder_inputs))


encoder_lstm = GRU(units=units, return_sequences=True, return_state=True)
encoder_out, state_h= encoder_lstm(encoder_emb(encoder_inputs))
# state_h = Concatenate()([fstate_h,bstate_h])
# state_c = Concatenate()([bstate_h,bstate_c])
encoder_states = state_h


# Now create the Decoder layers.
decoder_inputs = Input(shape=(None,))
decoder_emb = Embedding(input_dim=vocab_out_size, output_dim=embedding_dim)
decoder_lstm = GRU(units=units, return_sequences=True, return_state=True)
decoder_lstm_out, _= decoder_lstm(decoder_emb(decoder_inputs), initial_state=encoder_states)
# Two dense layers added to this model to improve inference capabilities.
decoder_d1 = Dense(units, activation="relu")
decoder_d2 = Dense(vocab_out_size, activation="softmax")
decoder_out = decoder_d2(Dropout(rate=.2)(decoder_d1(Dropout(rate=.2)(decoder_lstm_out))))


# Finally, create a training model which combines the encoder and the decoder.
# Note that this model has three inputs:
model = Model(inputs = [encoder_inputs, decoder_inputs], outputs= decoder_out)

# We'll use sparse_categorical_crossentropy so we don't have to expand decoder_out into a massive one-hot array.
# Adam is used because it's, well, the best.

model.compile(optimizer=tf.optimizers.Adam(), loss="sparse_categorical_crossentropy", metrics=['sparse_categorical_accuracy'])
model.summary()